In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

load_dotenv()

model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')


In [21]:
#this will work as schema

class Person(BaseModel):

    Name: str = Field (description= "Name of the person")
    Bith_Year : int = Field (gt= 18 , description= "Birth year of the person")
    city : str = Field(description= "city where the person was born")
    
    
parser = PydanticOutputParser(pydantic_object=Person)

# now about the template

template = PromptTemplate(
    input_variables = ['name'],
    template = "generate name, birthyear and city of a historical {name} \n {format_instruction}",
    partial_variables= {'format_instruction': parser.get_format_instructions()}
)

In [30]:
chain = template | model | parser



name = chain.invoke({'name': 'hitler'})


Person(Name='Adolf Hitler', Bith_Year=1889, city='Braunau am Inn')

In [31]:
def filldata (data):
    name = chain.invoke({'name': data})
    return name.city

In [23]:
import pandas as pd


data = {
    "Name": [
        "Leonardo da Vinci",
        "Rabindranath Tagore",
        "Nelson Mandela",
        "Cleopatra",
        "Albert Einstein"
    ]}

df = pd.DataFrame(data)


In [33]:
df['City'] = df['Name'].apply(filldata)

In [34]:
df

,Name,City
0,Leonardo da Vinci,Vinci
1,Rabindranath Tagore,Kolkata
2,Nelson Mandela,Mvezo
3,Cleopatra,Alexandria
4,Albert Einstein,Ulm
